In [1]:
import sys
sys.path.append('../src')

In [2]:
import json
import os
import numpy as np
import pandas as pd
from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader

from utils import count_commits, get_combined_df, tokenize, reverse_tokenize
from collections import defaultdict

[0.006s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016d07c000-0x000000016d088000).
[0.006s][warning][os,thread] Attempt to deallocate stack guard pages failed.


/opt/homebrew/Caskroom/miniconda/base/envs/ds/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class SearchResult:
    def __init__(self, commit_id, file_path, score, commit_date, commit_msg):
        self.commit_id = commit_id
        self.file_path = file_path
        self.score = score
        self.commit_date = commit_date
        self.commit_msg = commit_msg


    def __repr__(self):
        class_name = self.__class__.__name__
        return f"{class_name}(score={self.score:.5f}, file_path={self.file_path!r}, commit_id={self.commit_id!r}, commit_date={self.commit_date})"

    def is_actual_modified(self, actual_modified_files):
        return self.file_path in actual_modified_files

    @staticmethod
    def print_results(query, search_results, show_only_actual_modified=False):
        actual_modified_files = query['actual_files_modified']
        for i, result in enumerate(search_results):
            if show_only_actual_modified and not result.is_actual_modified(actual_modified_files):
                continue
            print(f"{i+1:2} {result}")

In [4]:
class AggregatedSearchResult:
    def __init__(self, file_path, aggregated_score, contributing_results):
        self.file_path = file_path
        self.score = aggregated_score
        self.contributing_results = contributing_results

    def __repr__(self):
        class_name = self.__class__.__name__
        return f"{class_name}(file_path={self.file_path!r}, score={self.score}, " \
               f"contributing_results={self.contributing_results})"

In [38]:
class BM25Search:
    def __init__(self, index_path):
        if not os.path.exists(index_path):
            raise FileNotFoundError(f"Index at {index_path} does not exist!")
        self.searcher = LuceneSearcher(index_path)
        print(f"Loaded index at {index_path}")
        print(f'Index Stats: {IndexReader(index_path).stats()}')
        # self.ranking_depth = ranking_depth

    def search(self, query, query_date, ranking_depth):
        # TODO maybe change this to mean returning reranking_depths total results instead of being pruned by the query date
        hits = self.searcher.search(tokenize(query), ranking_depth)
        unix_date = query_date
        filtered_hits = [
            SearchResult(hit.docid, json.loads(hit.raw)['file_path'], hit.score, int(json.loads(hit.raw)["commit_date"]), reverse_tokenize(json.loads(hit.raw)['contents']))
            for hit in hits if int(json.loads(hit.raw)["commit_date"]) < unix_date
        ]
        return filtered_hits

    def search_full(self, query, query_date, ranking_depth):
        filtered_hits = []
        step_size = ranking_depth  # Initial search window
        total_hits_retrieved = 0

        while len(filtered_hits) < ranking_depth and step_size > 0:
            current_hits = self.searcher.search(tokenize(query), total_hits_retrieved + step_size)
            if not current_hits:
                break  # No more results to retrieve

            # Filter hits by query date
            for hit in current_hits:
                if int(json.loads(hit.raw)["commit_date"]) < query_date:
                    filtered_hits.append(
                        SearchResult(hit.docid, json.loads(hit.raw)['file_path'], hit.score,
                                     int(json.loads(hit.raw)["commit_date"]),
                                     reverse_tokenize(json.loads(hit.raw)['contents']))
                    )
                if len(filtered_hits) == ranking_depth:
                    break  # We have enough results

            total_hits_retrieved += step_size
            step_size = ranking_depth - len(filtered_hits)  # Decrease step size to only get as many as needed

        return filtered_hits[:ranking_depth]  # Return up to ranking_depth results

    def aggregate_file_scores(self, search_results, aggregation_method='sump'):
        file_to_results = defaultdict(list)
        for result in search_results:
            file_to_results[result.file_path].append(result)

        aggregated_results = []
        for file_path, results in file_to_results.items():
            # aggregated_score = sum(result.score for result in results)
            if aggregation_method == 'sump':
                aggregated_score = sum(result.score for result in results)
            elif aggregation_method == 'maxp':
                aggregated_score = max(result.score for result in results)
            # elif aggregation_method == 'firstp':
            #     aggregated_score = results[0].score
            elif aggregation_method == 'avgp':
                aggregated_score = np.mean([result.score for result in results])
            else:
                raise ValueError(f"Unknown aggregation method {aggregation_method}")

            aggregated_results.append(AggregatedSearchResult(file_path, aggregated_score, results))

        aggregated_results.sort(key=lambda result: result.score, reverse=True)
        return aggregated_results

    def pipeline(self, query, query_date, ranking_depth, aggregation_method):
        search_results = self.search(query, query_date, ranking_depth)
        if aggregation_method is not None:
            aggregated_results = self.aggregate_file_scores(search_results, aggregation_method)
            return aggregated_results
        return search_results

In [37]:
class SearchEvaluator:
    def __init__(self, metrics):
        self.metrics = metrics

    @staticmethod
    def precision_at_k(relevant, k):
        return sum(relevant[:k]) / k

    @staticmethod
    def mean_reciprocal_rank(relevant):
        for idx, value in enumerate(relevant):
            if value == 1:
                return 1 / (idx + 1)
        return 0

    @staticmethod
    def calculate_average_precision(relevant):
        pred_rel = [1] * len(relevant)
        relevant_documents_count = 0
        cumulative_precision = 0.0

        # We iterate through the predicted relevance scores
        for i in range(len(pred_rel)):
            # Check if the prediction at this rank is correct (i.e., if it is a relevant document)
            if pred_rel[i] == 1 and relevant[i] == 1:
                relevant_documents_count += 1
                precision_at_i = relevant_documents_count / (i + 1)
                cumulative_precision += precision_at_i

        # The average precision is the cumulative precision divided by the number of relevant documents
        average_precision = cumulative_precision / sum(relevant) if sum(relevant) > 0 else 0
        return average_precision

    # @staticmethod
    # def calculate_recall(relevant, total_modified_files, k):
    #   # Does not work for commit based approach as it can have multiple mentions of the same file across commits leading to a higher than 1 recall
    #     print(total_modified_files)
    #     print(relevant)
    #     return sum(relevant[:k]) / total_modified_files

    @staticmethod
    def calculate_recall(retrieved_files, actual_modified_files, relevant, k):
        # this complicated mess is required as compared to the above much simpler code to support both commit-based and file-based approaches
        # in file-based approach, this is equivalent to the above code
        # in code-based approach, duplicates could be present in retrieved_files, which is why we need to filter them out (the above code would not work in this case)

        return len({file for idx, file in enumerate(retrieved_files[:k])
                        if relevant[idx] == 1
                    }) / len(actual_modified_files) if len(actual_modified_files) > 0 else 0


    def evaluate(self, search_results, actual_modified_files):
        retrieved_files = [result.file_path for result in search_results]
        relevant = [1 if file in actual_modified_files else 0 for file in retrieved_files]


        evaluations = {}
        for metric in self.metrics:
            if metric == 'MAP':
                evaluations[metric] = self.calculate_average_precision(relevant)
            elif metric == 'MRR':
                evaluations[metric] = self.mean_reciprocal_rank(relevant)
            elif metric.startswith('P@'):
                k = int(metric.split('@')[1])
                evaluations[metric] = self.precision_at_k(relevant, k)
            elif metric.startswith('Recall@'):
                k = int(metric.split('@')[1])

                # evaluations[metric] = len(list({
                #         file
                #         for idx, file in enumerate(retrieved_files)
                #         if relevant[idx] == 1
                #     })[:k]) / len(actual_modified_files)
                # evaluations[metric] = self.calculate_recall(relevant, len(actual_modified_files), k) # DOES NOT WORK FOR COMMIT-BASED APPROACH

                evaluations[metric] = self.calculate_recall(retrieved_files, actual_modified_files, relevant, k)


        return {k: round(v, 4) for k, v in evaluations.items()}

    # def evaluate_file_based(self, search_results, actual_modified_files, aggregation_strategy='sump'):
    #     file_relevance = defaultdict(list)

    #     # Aggregate relevance scores for each file across all commits
    #     for result in search_results:
    #         if result.file_path in actual_modified_files:
    #             # file_relevance[result.file_path] += 1
    #             file_relevance[result.file_path].append(result.score)

    #     # Normalize relevance scores based on occurrences in actual modified files
    #     # max_relevance = max(file_relevance.values(), default=1)
    #     # normalized_relevance = {file: relevance / max_relevance for file, relevance in file_relevance.items()}
    #     # sorted_normalized_relevance = sorted(normalized_relevance.items(), key=lambda item: item[1], reverse=True)
    #     print(file_relevance)
    #     if aggregation_strategy == 'sump':
    #         aggregated_scores = {file: sum(relevance) for file, relevance in file_relevance.items()}
    #     elif aggregation_strategy == 'firstp':
    #         aggregated_scores = {file: relevance[0] for file, relevance in file_relevance.items()}
    #     elif aggregation_strategy == 'avgp':
    #         aggregated_scores = {file: np.mean(relevance) for file, relevance in file_relevance.items()}
    #     else:
    #         raise ValueError(f"Unknown aggregation strategy {aggregation_strategy}")

    #     sorted_aggregated_scores = sorted(aggregated_scores.items(), key=lambda item: item[1], reverse=True)

    #     print(sorted_aggregated_scores)

    #     evaluations = {}
    #     for metric in self.metrics:
    #         if metric.startswith('P@'):
    #             # Compute precision at k for files, not individual commit mentions
    #             k = int(metric.split('@')[1])
    #             # top_k_files = sorted(normalized_relevance.items(), key=lambda item: item[1], reverse=True)[:k]
    #             top_k_files = sorted_aggregated_scores[:k]
    #             precision_at_k = sum(1 for file, relevance in top_k_files if file in actual_modified_files) / k
    #             evaluations[metric] = precision_at_k
    #         elif metric.startswith('Recall@'):
    #             k = int(metric.split('@')[1])
    #             # top_k_files = sorted(normalized_relevance.items(), key=lambda item: item[1], reverse=True)[:k]
    #             top_k_files = sorted_aggregated_scores[:k]
    #             recall_at_k = sum(1 for file, relevance in top_k_files if file in actual_modified_files) / len(actual_modified_files)
    #             evaluations[metric] = recall_at_k
    #         elif metric == 'MAP':
    #             # Compute average precision for files, not individual commit mentions
    #             average_precision = 0
    #             num_relevant_files = 0
    #             for idx, (file, relevance) in enumerate(sorted_aggregated_scores):
    #                 if file in actual_modified_files:
    #                     num_relevant_files += 1
    #                     average_precision += num_relevant_files / (idx + 1)
    #             average_precision /= len(actual_modified_files)
    #             evaluations[metric] = average_precision
    #         elif metric == 'MRR':
    #             # Compute mean reciprocal rank for files, not individual commit mentions
    #             reciprocal_rank = 0
    #             for idx, (file, relevance) in enumerate(sorted_aggregated_scores):
    #                 if file in actual_modified_files:
    #                     reciprocal_rank = 1 / (idx + 1)
    #                     break
    #             evaluations[metric] = reciprocal_rank

    #     return {k: round(v, 4) for k, v in evaluations.items()}

In [40]:
class ModelEvaluator:
    def __init__(self, model, eval_model, combined_df, seed=42):
        self.model = model
        self.eval_model = eval_model
        self.combined_df = combined_df
        self.seed = seed

    def sample_commits(self, n):
        if self.combined_df.commit_id.nunique() < n:
            raise ValueError(f'Not enough commits to sample. Required: {n}, available: {self.combined_df.commit_id.nunique()}')
        return self.combined_df.drop_duplicates(subset='commit_id').sample(n=n, replace=False, random_state=self.seed)

    def evaluate_sampling(self, n=100, k=1000, output_dir='.', skip_existing=False, evaluation_strategy='commit', aggregation_strategy=None):
        model_name = self.model.__class__.__name__
        output_file = f"{output_dir}/{model_name}_metrics.txt"

        if skip_existing and os.path.exists(output_file):
            print(f'Output file {output_file} already exists, skipping...')
            return

        sampled_commits = self.sample_commits(n)

        results = []
        for _, row in sampled_commits.iterrows():
            # search_results = self.model.search(row['commit_message'], row['commit_date'], ranking_depth=k)
            # TODO: Add ChatGPT based query modification here
            search_results = self.model.pipeline(row['commit_message'], row['commit_date'], ranking_depth=k, aggregation_method=aggregation_strategy)
            # if evaluation_strategy == 'commit':
            evaluation = self.eval_model.evaluate(search_results,
                                                       self.combined_df[self.combined_df['commit_id'] == row['commit_id']]['file_path'].tolist())
            # elif evaluation_strategy == 'file':
            #     evaluation = self.eval_model.evaluate_file_based(search_results,
            #                                                       self.combined_df[self.combined_df['commit_id'] == row['commit_id']]['file_path'].tolist(), aggregation_strategy=aggregation_strategy)
            # else:
            #     raise ValueError(f'Invalid evaluation strategy: {evaluation_strategy}')
            results.append(evaluation)

        avg_scores = {metric: round(np.mean([result[metric] for result in results]), 4) for metric in results[0]}

        os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
        with open(output_file, "w") as file:
            file.write(f"Model Name: {model_name}\n")
            file.write(f"Sample Size: {n}\n")
            file.write("Evaluation Metrics:\n")
            for key, value in avg_scores.items():
                file.write(f"{key}: {value}\n")

        return avg_scores

In [186]:
def tmp():
    def calculate_recall(relevant, total_modified_files, k):
        return sum(relevant[:k]) / total_modified_files

    def calculate_average_precision(relevant):
        pred_rel = [1] * len(relevant)
        relevant_documents_count = 0
        cumulative_precision = 0.0

        # We iterate through the predicted relevance scores
        for i in range(len(pred_rel)):
            # Check if the prediction at this rank is correct (i.e., if it is a relevant document)
            if pred_rel[i] == 1 and relevant[i] == 1:
                relevant_documents_count += 1
                precision_at_i = relevant_documents_count / (i + 1)
                cumulative_precision += precision_at_i

        # The average precision is the cumulative precision divided by the number of relevant documents
        average_precision = cumulative_precision / sum(relevant)
        return average_precision

    def mean_reciprocal_rank(relevant):
        for idx, value in enumerate(relevant):
            if value == 1:
                return 1 / (idx + 1)
        return 0

    def precision_at_k(relevant, k):
        return sum(relevant[:k]) / k

    rel = [0,1,0,1,0,1,0,0,1]
    k = 3
    # print(calculate_recall(rel, 4, k))
    print(f'MAP: {calculate_average_precision(rel)}')
    print(f'MRR: {mean_reciprocal_rank(rel)}')

    for k in range(1, 10):
        print(f'P@{k}: {precision_at_k(rel, k)}')
        print(f'Recall@{k}: {calculate_recall(rel, sum(rel), k)}')

tmp()

MAP: 0.4861111111111111
MRR: 0.5
P@1: 0.0
Recall@1: 0.0
P@2: 0.5
Recall@2: 0.25
P@3: 0.3333333333333333
Recall@3: 0.25
P@4: 0.5
Recall@4: 0.5
P@5: 0.4
Recall@5: 0.5
P@6: 0.5
Recall@6: 0.75
P@7: 0.42857142857142855
Recall@7: 0.75
P@8: 0.375
Recall@8: 0.75
P@9: 0.4444444444444444
Recall@9: 1.0


In [9]:
index_path = '../smalldata/fbr/index_commit_tokenized'
repo_path = '../smalldata/fbr/'
K=1000

In [10]:
metrics = ['MAP', 'P@10', 'P@100', 'P@1000', 'MRR', 'Recall@100', 'Recall@1000']

In [11]:
combined_df = get_combined_df(repo_path)

In [41]:
bm25_searcher = BM25Search(index_path)
evaluator = SearchEvaluator(metrics)
bm25_evaluator = ModelEvaluator(bm25_searcher, evaluator, combined_df)

Loaded index at ../smalldata/fbr/index_commit_tokenized
Index Stats: {'total_terms': 8061856, 'documents': 69835, 'non_empty_documents': 69835, 'unique_terms': 14589}


In [13]:
# randomly sample a commit from the combined_df
random_commit = combined_df.sample(1).iloc[0]
random_commit

owner                                                             facebook
repo_name                                                            react
commit_date                                                     1522876866
commit_id                         27535e7bfcb63e8a4d65f273311e380b4ca12eff
commit_message           Clarify ReactDOM's case warning for html tags ...
file_path                packages/react-dom/src/server/ReactPartialRend...
cur_file_content         /**\n * Copyright (c) 2013-present, Facebook, ...
previous_commit_id                8ec0e4a99df76c0ff1779cac4f2eaaaf35a6b5bb
previous_file_path                                                    <NA>
previous_file_content    /**\n * Copyright (c) 2013-present, Facebook, ...
diff                     @@ -955,8 +955,9 @@ class ReactDOMServerRender...
status                                                            modified
is_merge_request                                                     False
file_extension           

In [14]:
# get search results for the random commit
search_results = bm25_searcher.search(random_commit['commit_message'], random_commit['commit_date'], ranking_depth=K)
search_results[:20]

[SearchResult(score=67.77170, file_path='src/renderers/dom/fiber/ReactDOMFiberEntry.js', commit_id='1f74eca9937ad6f19b6291d21edfb8747bae88ca', commit_date=1500652126),
 SearchResult(score=67.77170, file_path='src/renderers/dom/fiber/__tests__/ReactDOMFiber-test.js', commit_id='1f74eca9937ad6f19b6291d21edfb8747bae88ca', commit_date=1500652126),
 SearchResult(score=67.77170, file_path='src/renderers/dom/shared/__tests__/ReactDOMTextComponent-test.js', commit_id='1f74eca9937ad6f19b6291d21edfb8747bae88ca', commit_date=1500652126),
 SearchResult(score=67.77170, file_path='src/renderers/dom/shared/__tests__/ReactMount-test.js', commit_id='1f74eca9937ad6f19b6291d21edfb8747bae88ca', commit_date=1500652126),
 SearchResult(score=67.77169, file_path='src/renderers/shared/fiber/ReactFiberReconciler.js', commit_id='1f74eca9937ad6f19b6291d21edfb8747bae88ca', commit_date=1500652126),
 SearchResult(score=67.77169, file_path='src/renderers/shared/fiber/ReactFiberTreeReflection.js', commit_id='1f74eca99

In [15]:
aggregated_results = bm25_searcher.aggregate_file_scores(search_results)
aggregated_results[:20]

[AggregatedSearchResult(file_path='src/renderers/shared/fiber/ReactChildFiber.js', score=245.6372833251953, contributing_results=[SearchResult(score=62.60210, file_path='src/renderers/shared/fiber/ReactChildFiber.js', commit_id='90294ead4c627715cb70f20ff448bb0d34ee4c1b', commit_date=1484003358), SearchResult(score=50.19460, file_path='src/renderers/shared/fiber/ReactChildFiber.js', commit_id='309412d8b6d6ed0cfbb50a6be2f1a99cca36fea5', commit_date=1499967692), SearchResult(score=47.07320, file_path='src/renderers/shared/fiber/ReactChildFiber.js', commit_id='3e48422fdc2feaf741d768a6071a87adf2fc107b', commit_date=1492614304), SearchResult(score=43.78109, file_path='src/renderers/shared/fiber/ReactChildFiber.js', commit_id='e36b38c1cad22f7dff557736cf4b0280106e937e', commit_date=1481819818), SearchResult(score=41.98630, file_path='src/renderers/shared/fiber/ReactChildFiber.js', commit_id='3b43f3190bbcc5d4cfe77dbc665e03b7b95d35b2', commit_date=1499899781)]),
 AggregatedSearchResult(file_path

In [16]:
len(search_results), len(aggregated_results)

(534, 428)

In [33]:
# evaluate the search results
evaluation = evaluator.evaluate(search_results, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist())
evaluation

{'MAP': 0.0097,
 'P@10': 0.0,
 'P@100': 0.0,
 'P@1000': 0.008,
 'MRR': 0.0035,
 'Recall@100': 0.0,
 'Recall@1000': 0.75}

In [26]:
file_based_evaluation = evaluator.evaluate(aggregated_results, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist())
file_based_evaluation

{'MAP': 0.0052,
 'P@10': 0.0,
 'P@100': 0.0,
 'P@1000': 0.003,
 'MRR': 0.003,
 'Recall@100': 0.0,
 'Recall@1000': 0.75}

Full Search Experiments

In [124]:
# search_full = bm25_searcher.search_full(random_commit['commit_message'], random_commit['commit_date'], ranking_depth=K)
# search_full[:20]

In [123]:
# aggregated_results_full = bm25_searcher.aggregate_file_scores(search_full)
# aggregated_results_full[:20]

In [122]:
# len(search_full), len(aggregated_results_full)

In [127]:
# evaluation_full = evaluator.evaluate(search_full, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist())
# evaluation_full

In [126]:
# file_based_evaluation_full = evaluator.evaluate(aggregated_results_full, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist())
# file_based_evaluation_full

In [34]:
# test different aggregation strategies in bm25 search
agggreagtion_strategies = ['sump', 'maxp', 'avgp']
for strategy in agggreagtion_strategies:
    print(f"Aggregation Strategy: {strategy}")
    aggregated_results = bm25_searcher.aggregate_file_scores(search_results, aggregation_method=strategy)
    evaluation = evaluator.evaluate(aggregated_results, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist())
    print(evaluation)

Aggregation Strategy: sump
{'MAP': 0.0674, 'P@10': 0.0, 'P@100': 0.03, 'P@1000': 0.003, 'MRR': 0.0588, 'Recall@100': 0.75, 'Recall@1000': 0.75}
Aggregation Strategy: maxp
{'MAP': 0.0062, 'P@10': 0.0, 'P@100': 0.0, 'P@1000': 0.003, 'MRR': 0.004, 'Recall@100': 0.0, 'Recall@1000': 0.75}
Aggregation Strategy: avgp
{'MAP': 0.0052, 'P@10': 0.0, 'P@100': 0.0, 'P@1000': 0.003, 'MRR': 0.003, 'Recall@100': 0.0, 'Recall@1000': 0.75}


In [42]:
bm25_evaluator.evaluate_sampling(n=100, k=K, output_dir='../tmp/')

{'MAP': 0.1794,
 'P@10': 0.079,
 'P@100': 0.0327,
 'P@1000': 0.0084,
 'MRR': 0.2676,
 'Recall@100': 0.4512,
 'Recall@1000': 0.6351}

In [36]:
for strategy in agggreagtion_strategies:
    print(f"Aggregation Strategy: {strategy}")
    results = bm25_evaluator.evaluate_sampling(n=100, k=K, output_dir='../tmp/', evaluation_strategy='file', aggregation_strategy=strategy)
    print(results)

Aggregation Strategy: sump
{'MAP': 0.2037, 'P@10': 0.074, 'P@100': 0.0202, 'P@1000': 0.0027, 'MRR': 0.2746, 'Recall@100': 0.4826, 'Recall@1000': 0.6351}
Aggregation Strategy: maxp
{'MAP': 0.2482, 'P@10': 0.06, 'P@100': 0.0181, 'P@1000': 0.0027, 'MRR': 0.2688, 'Recall@100': 0.4629, 'Recall@1000': 0.6351}
Aggregation Strategy: avgp
{'MAP': 0.1773, 'P@10': 0.048, 'P@100': 0.0149, 'P@1000': 0.0027, 'MRR': 0.1968, 'Recall@100': 0.4011, 'Recall@1000': 0.6351}


In [50]:
# file_based_evaluation = evaluator.evaluate_file_based(search_results, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist())
# file_based_evaluation

In [51]:
# # iterate over the list of aggregation strategies and evaluate each one

# aggregation_strategies = ['sump', 'maxp', 'firstp', 'avgp']
# for strategy in aggregation_strategies:
#     file_based_evaluation = evaluator.evaluate_file_based(search_results, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist(), aggregation_strategy=strategy)
#     print(f"{strategy}: {file_based_evaluation}")

In [52]:
# # do the same for the model evaluator
# for strategy in aggregation_strategies:
#     model_evaluation = bm25_evaluator.evaluate_sampling(n=1000, k=K, output_dir='../tmp/', evaluation_strategy='file', aggregation_strategy=strategy)
#     print(f"{strategy}: {model_evaluation}")

In [127]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from collections import defaultdict

class BERTReRanker:
    """
    A class for performing reranking with a BERT-based model.
    """
    def __init__(self, model_name, psgLen=128, psgStride=64, psgCnt=None, scoreAggregation='maxp', batchSize=8):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
        self.model.eval()  # Set model to evaluation mode

        # Passage handling parameters
        self.psgLen = psgLen
        self.psgStride = psgStride
        self.psgCnt = psgCnt
        self.scoreAggregation = scoreAggregation

        if self.scoreAggregation == 'firstp':
            self.psgCnt = 1

        self.batchSize = batchSize

    def rerank(self, query, search_results):
        """
        Rerank the search results using the BERT model.

        query: The query string.
        search_results: A list of SearchResult objects.
        """
        reranked_results = []
        # todo - add batching

        # Process each SearchResult to create input for BERT
        for result in search_results:
            passages = self._split_into_passages(query, result.commit_msg)

            # Score each passage with BERT
            passage_scores = [self._score_passage(query, passage) for passage in passages]

            # Aggregate passage scores to get a single document score
            doc_score = self._aggregate_scores(passage_scores)

            # Create a new SearchResult with the updated score
            reranked_results.append((doc_score, result))

        # Sort reranked results by the new score
        reranked_results.sort(key=lambda x: x[0], reverse=True)
        return [result for _, result in reranked_results]

    def _split_into_passages(self, query, commit_msg):
        # Tokenize the query and commit message
        tokens = self.tokenizer.tokenize(query) + self.tokenizer.tokenize(commit_msg)

        # Split the tokens into passages
        passages = []
        for i in range(0, len(tokens), self.psgStride):
            passage = tokens[i:i+self.psgLen]
            passages.append(self.tokenizer.convert_tokens_to_string(passage))
            if self.psgCnt and len(passages) >= self.psgCnt:
                break
        return passages

    def _score_passage(self, query, passage):
        # Encode query and passage for BERT
        inputs = self.tokenizer.encode_plus(
            query,
            passage,
            add_special_tokens=True,
            max_length=self.psgLen,
            truncation="only_second",
            return_tensors="pt"
        )

        # Score the (query, passage) pair with BERT
        with torch.no_grad():
            outputs = self.model(**inputs)
            score = outputs.logits.squeeze().item()
        return score

    def _aggregate_scores(self, passage_scores):
        # Aggregate passage scores based on the specified strategy
        if self.scoreAggregation == 'firstp':
            return passage_scores[0]
        elif self.scoreAggregation == 'maxp':
            return max(passage_scores)
        elif self.scoreAggregation == 'avgp':
            return sum(passage_scores) / len(passage_scores)
        elif self.scoreAggregation == 'sump':
            return sum(passage_scores)
        else:
            raise ValueError(f"Invalid score aggregation method: {self.scoreAggregation}")

In [128]:
# Assuming bm25_searcher is an instance of BM25Search
query = "Fix login page error"
query_date = 1699261235
bm25_results = bm25_search.search(query, query_date, K)

# Now rerank those results with BERT
bert_reranker = BERTReRanker(model_name="bert-base-uncased")
reranked_results = bert_reranker.rerank(query, bm25_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (791 > 512). Running this sequence through the model will result in indexing errors


In [130]:
# Print the top 10 results
reranked_results[:10]

[SearchResult(score: 2.69137, file_path: 'packages/react-reconciler/src/ReactFiberBeginWork.new.js', commit_id: '1faf9e3dd5d6492f3607d5c721055819e4106bc6', commit_date: 1601495853),
 SearchResult(score: 2.69136, file_path: 'packages/react-reconciler/src/ReactFiberBeginWork.old.js', commit_id: '1faf9e3dd5d6492f3607d5c721055819e4106bc6', commit_date: 1601495853),
 SearchResult(score: 2.69136, file_path: 'packages/react-reconciler/src/ReactFiberSuspenseComponent.new.js', commit_id: '1faf9e3dd5d6492f3607d5c721055819e4106bc6', commit_date: 1601495853),
 SearchResult(score: 2.69136, file_path: 'packages/react-reconciler/src/ReactFiberSuspenseComponent.old.js', commit_id: '1faf9e3dd5d6492f3607d5c721055819e4106bc6', commit_date: 1601495853),
 SearchResult(score: 2.69136, file_path: 'packages/react-reconciler/src/__tests__/ReactCPUSuspense-test.js', commit_id: '1faf9e3dd5d6492f3607d5c721055819e4106bc6', commit_date: 1601495853),
 SearchResult(score: 3.26617, file_path: 'packages/react-dom-bindi

In [131]:
# evaluate the reranked results
evaluation = evaluator.evaluate(reranked_results, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist())
evaluation

{'MAP': 0.016,
 'P@10': 0.0,
 'P@100': 0.0,
 'P@1000': 0.016,
 'MRR': 0.0031,
 'Recall@1000': 0.3913}

In [137]:
evaluation = evaluator.evaluate_file_based(reranked_results, combined_df[combined_df['commit_id'] == random_commit['commit_id']]['file_path'].tolist())
evaluation

defaultdict(<class 'list'>, {'packages/react-reconciler/src/ReactFiberCompleteWork.js': [2.718986988067627, 3.8634390830993652, 3.863529920578003, 4.273975372314453], 'packages/shared/ReactSymbols.js': [3.5164780616760254], 'packages/shared/ReactTypes.js': [3.6565749645233154, 3.8633739948272705, 3.863521099090576], 'scripts/rollup/bundles.js': [4.612883567810059, 5.564578056335449, 2.7962939739227295], 'packages/react-dom/src/shared/assertValidProps.js': [5.564676284790039], 'packages/react-dom/src/events/DOMEventResponderSystem.js': [3.8635449409484863], 'packages/react-dom/src/events/__tests__/DOMEventResponderSystem-test.internal.js': [3.86354398727417], 'packages/react-events/src/FocusScope.js': [3.863539934158325], 'packages/shared/getComponentName.js': [3.8635189533233643]})
[('packages/react-reconciler/src/ReactFiberCompleteWork.js', 14.719931364059448), ('scripts/rollup/bundles.js', 12.973755598068237), ('packages/shared/ReactTypes.js', 11.383470058441162), ('packages/react-do

{'MAP': 0.3913,
 'P@10': 0.9,
 'P@100': 0.09,
 'P@1000': 0.009,
 'MRR': 1.0,
 'Recall@1000': 0.3913}

In [ ]:
{'MAP': 0.0546,
 'P@10': 0.0,
 'P@100': 0.05,
 'P@1000': 0.019,
 'MRR': 0.0119,
 'Recall@1000': 0.3478}

{'MAP': 0.3478,
 'P@10': 0.8,
 'P@100': 0.08,
 'P@1000': 0.008,
 'MRR': 1.0,
 'Recall@1000': 0.3478}